In [1]:
import os
import torch
import numpy as np
import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))
from Utils.context_fid import Context_FID
from Utils.metric_utils import display_scores
from Utils.cross_correlation import CrossCorrelLoss

In [2]:
ori_data = np.load('../Data/samples_24/sine_ground_truth.npy')
fake_data = np.load('../Data/ldm_fake_SINE.npy')
iterations = 5

In [3]:
## Context-FID Score
context_fid_score = []
for i in range(iterations):
    context_fid = Context_FID(ori_data[:], fake_data[:])
    context_fid_score.append(context_fid)
    print(f'Iter {i}: ', 'context-fid =', context_fid, '\n')
      
display_scores(context_fid_score)

Iter 0:  context-fid = 0.0018362063659137207 

Iter 1:  context-fid = 0.0020714387536055977 

Iter 2:  context-fid = 0.001638366527255291 

Iter 3:  context-fid = 0.002035276054623969 

Iter 4:  context-fid = 0.002007299401145911 

Final Score:  0.0019177174205088979 ± 0.0002239785864939179


In [4]:
## Correlational Score
def random_choice(size, num_select=100):
    select_idx = np.random.randint(low=0, high=size, size=(num_select,))
    return select_idx

x_real = torch.from_numpy(ori_data)
x_fake = torch.from_numpy(fake_data)

correlational_score = []
size = int(x_real.shape[0] / iterations)

for i in range(iterations):
    real_idx = random_choice(x_real.shape[0], size)
    fake_idx = random_choice(x_fake.shape[0], size)
    corr = CrossCorrelLoss(x_real[real_idx, :, :], name='CrossCorrelLoss')
    loss = corr.compute(x_fake[fake_idx, :, :])
    correlational_score.append(loss.item())
    print(f'Iter {i}: ', 'cross-correlation =', loss.item(), '\n')

display_scores(correlational_score)

Iter 0:  cross-correlation = 0.020529337881655813 

Iter 1:  cross-correlation = 0.0225930008782532 

Iter 2:  cross-correlation = 0.008389008212393589 

Iter 3:  cross-correlation = 0.02142322274780256 

Iter 4:  cross-correlation = 0.016348219082228728 

Final Score:  0.01785655776046678 ± 0.00719223423997651


In [5]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import sys
sys.path.append(os.path.join(os.path.dirname('__file__'), '../'))

import warnings
warnings.filterwarnings("ignore")

import numpy as np
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

from Utils.metric_utils import display_scores
from Utils.discriminative_metric import discriminative_score_metrics
from Utils.predictive_metric import predictive_score_metrics

In [6]:
### Discriminative score
discriminative_score = []

for i in range(iterations):
    temp_disc, fake_acc, real_acc = discriminative_score_metrics(ori_data[:], fake_data[:])
    discriminative_score.append(temp_disc)
    print(f'Iter {i}: ', temp_disc, ',', fake_acc, ',', real_acc, '\n')
      
print('sine:')
display_scores(discriminative_score)
print()

Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Please use tf.global_variables instead.


training: 100%|██████████| 2000/2000 [00:24<00:00, 80.22it/s]


Iter 0:  0.0030000000000000027 , 0.4555 , 0.5505 



training: 100%|██████████| 2000/2000 [00:25<00:00, 77.67it/s]


...

Iter 2:  0.0 , 0.4325 , 0.5675 



training: 100%|██████████| 2000/2000 [00:25<00:00, 79.92it/s]


Iter 3:  0.00824999999999998 , 0.682 , 0.3015 



training: 100%|██████████| 2000/2000 [00:25<00:00, 79.98it/s]

Iter 4:  0.005750000000000033 , 0.41 , 0.6015 

sine:
Final Score:  0.004499999999999993 ± 0.003883302913710611



In [7]:
### Predictive score
predictive_score = []
for i in range(iterations):
    temp_pred = predictive_score_metrics(ori_data, fake_data[:])
    predictive_score.append(temp_pred)
    print(i, ' epoch: ', temp_pred, '\n')
      
print('sine:')
display_scores(predictive_score)
print()

training: 100%|██████████| 5000/5000 [00:44<00:00, 113.49it/s]


0  epoch:  0.0930039014636426 



training: 100%|██████████| 5000/5000 [00:43<00:00, 115.46it/s]


1  epoch:  0.09299027901968789 



training: 100%|██████████| 5000/5000 [00:44<00:00, 113.15it/s]


2  epoch:  0.09293339668731854 



training: 100%|██████████| 5000/5000 [00:43<00:00, 114.46it/s]


3  epoch:  0.09300780699598002 



training: 100%|██████████| 5000/5000 [00:43<00:00, 115.90it/s]


4  epoch:  0.09287414435235639 

sine:
Final Score:  0.0929619057037971 ± 7.130232225332479e-05

